In [5]:
import tensorflow as tf
import psycopg2 as pg
import configparser

print(tf.__version__)

2.10.0


In [24]:
config.read('config.ini')

host = config['db_connect']['host']
port = config['db_connect']['port']
user = config['db_connect']['user']
password = config['db_connect']['pass']
dbname = config['db_connect']['dbname']

In [25]:
pg.connect(host=host,
           port=port,
           user=user,
           password=password,
           dbname=dbname)

OperationalError: could not translate host name "webdev.iticscppi.com" to address: Unknown host
